In [2]:
import blimpy

In [3]:
FILENAME = "data/blc41_guppi_59103_01372_PSR_B2021+51_0009.rawspec.0000.h5"

In [4]:
fb = blimpy.Waterfall(FILENAME)
fb.data.shape
fb.data[5][0][30]

13690.115

In [5]:
import h5py

In [14]:
f = h5py.File(FILENAME, "r")
f["data"][5][0][30]

13690.115